<a href="https://colab.research.google.com/github/WallaceSantana79/CursoMachineLearning/blob/main/Aprendizagem_por_refor%C3%A7o_com_Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Aprendizagem por reforço com Q-Learning

In [1]:
!pip uninstall gym

Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/gym-0.25.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/gym/*
Proceed (Y/n)? y
  Successfully uninstalled gym-0.25.2


In [2]:
!pip install gym==0.15.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.15.7-py3-none-any.whl size=1648805 sha256=5e0c079dbaf23652ad7d5f39a6c18f8a045fea91b870ad7b195f8e7a66812a72
  Stored in directory: /root/.cache/pip/wheels/09/34/69/e1c9061afff9529cff995b1861b5e88c31b845dbb28ea1c9d6
Successfully built gym
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.1
    Uninstalling cloudpickle-2.2.1:
      Successfully uninstalled cloudpickle-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.17.0 requires cloudpickle>=2.0.0, but you have cloudpickle 1.2.2 which is incompatible.
dask 2023.8.1 requires cloudpickle>=1.5.0, but you have cloudpickle 1.2.2 whic

##Entendimento do ambiente

In [3]:
import gym
import random
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [4]:
env = gym.make('Taxi-v3')

In [5]:
env.reset()

154

In [6]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [7]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [8]:
# 0 = south 1 = north 2 = east 3 = west 4 = pickup 5 = dropoff
print(env.action_space)

Discrete(6)


In [9]:
# 4 destinos
print(env.observation_space)

Discrete(500)


In [10]:
len(env.P)

500

In [11]:
env.P[484]

{0: [(1.0, 484, -1, False)],
 1: [(1.0, 384, -1, False)],
 2: [(1.0, 484, -1, False)],
 3: [(1.0, 464, -1, False)],
 4: [(1.0, 484, -10, False)],
 5: [(1.0, 484, -10, False)]}

##Treinamento

In [12]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table.shape

(500, 6)

In [13]:
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [14]:
np.argmax(np.array([3, 5]))

1

In [15]:
# 1-10% 3-90%
#exploration / exploitation
# 0 = south 1= north 2 = east 3 = west 4 = pickup 5 = dropoff
%%time
from IPython.display import clear_output

alpha = 0.1
gamma = 0.6
epsilon = 0.1

for i in range(1000000):
  estado = env.reset()

  penalidades, recompensa = 0,0
  done = False
  while not done:
    # Exploração
    if random.uniform(0,1) < epsilon:
      acao = env.action_space.sample()
      # Exploitation
    else:
      acao = np.argmax(q_table[estado])

    proximo_estado, recompensa, done, info = env.step(acao)

    q_antigo = q_table[estado,acao]
    proximo_maximo = np.max(q_table[proximo_estado])

    q_novo = (1-alpha) * q_antigo + alpha * (recompensa + gamma * proximo_maximo)
    q_table[estado,acao] = q_novo

    if recompensa == -10:
      penalidades += 1

    estado = proximo_estado

  if i % 100 == 0:
    clear_output(wait=True)
    print('Episódio: ',i)

print('Treinamento concluído')

Episódio:  999900
Treinamento concluído
CPU times: user 8min 32s, sys: 44 s, total: 9min 16s
Wall time: 9min 15s


In [23]:
q_table[429]

array([ -2.48941934,  -2.48236806,  -2.49364877,  -2.48941862,
       -11.48585443, -11.48851472])

In [19]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [22]:
env.encode(4,1,2,1)

429

##Avaliação

In [25]:
total_penalidades = 0
episodios = 50
frames = []

for _ in range(episodios):
  estado = env.reset()
  penalidades, recompensa = 0,0
  done = False
  while not done:
    acao = np.argmax(q_table[estado])
    estado, recompensa, done, info = env.step(acao)

    if recompensa == -10:
      penalidades += 1

    frames.append({
        'frame': env.render(mode='ansi'),
        'state': estado,
        'action': acao,
        'reward': recompensa
        })

    total_penalidades += penalidades

print('Episódios', episodios)
print('Penalidades', total_penalidades)

Episódios 50
Penalidades 0


In [27]:
from time import sleep
for frame in frames:
  clear_output(wait=True)
  print(frame['frame'])
  print('Estado', frame['state'])
  print('Ação', frame['action'])
  print('Recompensa', frame['reward'])
  sleep(.5)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Estado 410
Ação 5
Recompensa 20


KeyboardInterrupt: 